In [65]:
import pandas as pd
import pytz
from datetime import datetime, timedelta
from dateutil import tz

In [ ]:
df = pd.read_csv("data/important/ammonium.csv", parse_dates=['datumBeginMeting', 'datumEindeMeting'])

df.drop(columns=['datumEindeMeting'], inplace=True)

df.set_index('datumBeginMeting', inplace=True)
df = df.rename_axis('DateTimeIndex')

is_dst = pd.Series([False] * len(df.index), index=df.index)
tz = pytz.timezone("UTC")

for date in df.index:
    dst_shift = tz.localize(date, is_dst=True).dst()

    is_dst[date] = dst_shift != timedelta(0)

is_dst[df.index.duplicated(keep='first')] = False

new_index = pd.to_datetime(df.index, format='%d/%m/%Y %H:%M').tz_localize(
    pytz.timezone("UTC"),
    ambiguous=is_dst)

if new_index.duplicated().any():
    df.index = new_index
    df = df[~df.index.duplicated()]
else:
    df.index = new_index

df

In [68]:
df.to_csv("ammonium_utc.csv")